# 📊 Análisis TMT con Gráficas - Optimización GA

## Objetivo
Este notebook genera visualizaciones detalladas del TMT (Total Miscoordination Time) después de la optimización con Algoritmo Genético, incluyendo gráficas y tablas con los valores de TMT.

## Características
- 📈 Gráficas de TMT antes vs después de optimización
- 📊 Ranking de mejoras por escenario
- 🎯 Análisis de coordinación
- 📋 Tablas detalladas con valores de TMT
- 🔍 Identificación de mejores y peores casos

## Interpretación TMT
- **TMT = Total Miscoordination Time** (tiempo total de descoordinación)
- **Valores MÁS BAJOS = MEJOR coordinación**
- **Valores MÁS ALTOS = PEOR coordinación**
- **TMT Improvement = TMT_antes - TMT_después**
- **Valor POSITIVO = TMT disminuyó (MEJOR coordinación)**
- **Valor NEGATIVO = TMT aumentó (PEOR coordinación)**


In [ ]:
# %% Imports and Setup
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import seaborn as sns

# Set style for better plots
plt.style.use('default')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
sns.set_palette("husl")

# Setup paths
processed_dir = Path("/Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG/data/processed")
results_dir = Path("/Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG/results")
figures_dir = results_dir / "figures"
figures_dir.mkdir(parents=True, exist_ok=True)

print("✅ Imports and setup completed successfully")
print(f"📁 Working directory: {Path.cwd()}")
print(f"📊 Figures will be saved to: {figures_dir}")


In [ ]:
# %% Load TMT Analysis Data
def load_tmt_data():
    """Load TMT analysis data"""
    # Look for the most recent TMT analysis file
    tmt_files = list(processed_dir.glob("tmt_analysis_by_scenario_*.json"))
    
    if not tmt_files:
        print("❌ No TMT analysis file found!")
        return None
    
    latest_file = max(tmt_files, key=lambda x: x.stat().st_mtime)
    print(f"📂 Loading TMT data from: {latest_file}")
    
    with open(latest_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    return data['scenario_details']

# Load data
scenarios_data = load_tmt_data()

if scenarios_data:
    print(f"📊 Loaded data for {len(scenarios_data)} scenarios")
    
    # Create DataFrame for easier manipulation
    df = pd.DataFrame(scenarios_data)
    
    # Sort by improvement (best first)
    df = df.sort_values('tmt_improvement', ascending=False).reset_index(drop=True)
    
    # Display basic statistics
    print(f"\n📈 BASIC STATISTICS:")
    print(f"   • Total scenarios: {len(df)}")
    print(f"   • Scenarios improved: {len(df[df['tmt_improvement'] > 0])}")
    print(f"   • Scenarios degraded: {len(df[df['tmt_improvement'] < 0])}")
    print(f"   • Total TMT before: {df['tmt_before'].sum():.3f} seconds")
    print(f"   • Total TMT after: {df['tmt_after'].sum():.3f} seconds")
    print(f"   • Net TMT change: {df['tmt_improvement'].sum():+.3f} seconds")
    
    # Display first few rows
    print(f"\n📋 SAMPLE DATA:")
    display(df[['scenario_id', 'tmt_before', 'tmt_after', 'tmt_improvement', 'improvement_percentage']].head(10))
else:
    print("❌ Failed to load TMT data")


In [ ]:
# %% Create TMT Comparison Plot
def plot_tmt_comparison(df):
    """Create TMT before vs after comparison plot"""
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 12))
    
    # Get top 30 scenarios for better visualization
    top_30 = df.head(30)
    x = np.arange(len(top_30))
    width = 0.35
    
    # Plot 1: TMT Before vs After (Bar chart)
    bars1 = ax1.bar(x - width/2, top_30['tmt_before'], width, 
                   label='TMT Antes', color='#e74c3c', alpha=0.8)
    bars2 = ax1.bar(x + width/2, top_30['tmt_after'], width, 
                   label='TMT Después', color='#3498db', alpha=0.8)
    
    ax1.set_xlabel('Escenario')
    ax1.set_ylabel('TMT (segundos)')
    ax1.set_title('Comparación TMT: Antes vs Después de Optimización GA\n(Top 30 Escenarios)', 
                  fontsize=14, fontweight='bold')
    ax1.set_xticks(x[::3])  # Show every 3rd scenario label
    ax1.set_xticklabels([top_30.iloc[i]['scenario_id'] for i in range(0, len(top_30), 3)], 
                       rotation=45, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add value labels on bars for first 10 scenarios
    for i in range(min(10, len(top_30))):
        ax1.text(i - width/2, top_30.iloc[i]['tmt_before'] + 0.5, 
                f'{top_30.iloc[i]["tmt_before"]:.1f}', 
                ha='center', va='bottom', fontsize=8)
        ax1.text(i + width/2, top_30.iloc[i]['tmt_after'] + 0.5, 
                f'{top_30.iloc[i]["tmt_after"]:.1f}', 
                ha='center', va='bottom', fontsize=8)
    
    # Plot 2: TMT Improvement (Bar chart with colors)
    colors = ['#2ecc71' if imp > 0 else '#e74c3c' for imp in top_30['tmt_improvement']]
    bars3 = ax2.bar(x, top_30['tmt_improvement'], color=colors, alpha=0.7)
    
    ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
    ax2.set_xlabel('Escenario')
    ax2.set_ylabel('Mejora TMT (segundos)')
    ax2.set_title('Mejora/Degradación TMT por Escenario\n(Verde = Mejora, Rojo = Degradación)', 
                  fontsize=14, fontweight='bold')
    ax2.set_xticks(x[::3])
    ax2.set_xticklabels([top_30.iloc[i]['scenario_id'] for i in range(0, len(top_30), 3)], 
                       rotation=45, ha='right')
    ax2.grid(True, alpha=0.3)
    
    # Add improvement percentage labels for significant changes
    for i, (imp, bar) in enumerate(zip(top_30['tmt_improvement'], bars3)):
        if abs(imp) > 2:  # Only label significant changes
            ax2.text(i, imp + (0.2 if imp > 0 else -0.2), f'{imp:.1f}s', 
                    ha='center', va='bottom' if imp > 0 else 'top', 
                    fontsize=8, fontweight='bold')
    
    plt.tight_layout()
    
    # Save plot
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plot_file = figures_dir / f"tmt_comparison_{timestamp}.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"📊 TMT comparison plot saved: {plot_file}")
    return plot_file

if 'df' in locals() and not df.empty:
    plot_tmt_comparison(df)
else:
    print("❌ No data available for plotting")


In [ ]:
# %% Create Coordination Analysis Plot
def plot_coordination_analysis(df):
    """Create coordination percentage analysis plot"""
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Get top 25 scenarios for better visualization
    top_25 = df.head(25)
    
    # Plot 1: Coordination percentage comparison
    x = np.arange(len(top_25))
    width = 0.35
    
    bars1 = ax1.bar(x - width/2, top_25['coordination_pct_before'], width, 
                   label='Coordinación % Antes', color='#e67e22', alpha=0.8)
    bars2 = ax1.bar(x + width/2, top_25['coordination_pct_after'], width, 
                   label='Coordinación % Después', color='#9b59b6', alpha=0.8)
    
    ax1.set_xlabel('Escenario')
    ax1.set_ylabel('Porcentaje de Coordinación (%)')
    ax1.set_title('Coordinación: Antes vs Después de Optimización\n(Top 25 Escenarios)', 
                  fontweight='bold')
    ax1.set_xticks(x[::3])
    ax1.set_xticklabels([top_25.iloc[i]['scenario_id'] for i in range(0, len(top_25), 3)], 
                       rotation=45, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 100)
    
    # Plot 2: Coordination improvement
    coord_improvement = top_25['coordination_pct_after'] - top_25['coordination_pct_before']
    colors = ['#2ecc71' if imp > 0 else '#e74c3c' for imp in coord_improvement]
    
    bars3 = ax2.bar(x, coord_improvement, color=colors, alpha=0.7)
    
    ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
    ax2.set_xlabel('Escenario')
    ax2.set_ylabel('Mejora de Coordinación (%)')
    ax2.set_title('Mejora de Coordinación por Escenario\n(Verde = Mejora, Rojo = Degradación)', 
                  fontweight='bold')
    ax2.set_xticks(x[::3])
    ax2.set_xticklabels([top_25.iloc[i]['scenario_id'] for i in range(0, len(top_25), 3)], 
                       rotation=45, ha='right')
    ax2.grid(True, alpha=0.3)
    
    # Add improvement labels for significant changes
    for i, imp in enumerate(coord_improvement):
        if abs(imp) > 10:  # Only label significant improvements
            ax2.text(i, imp + (1 if imp > 0 else -1), f'{imp:.1f}%', 
                    ha='center', va='bottom' if imp > 0 else 'top', 
                    fontsize=8, fontweight='bold')
    
    plt.tight_layout()
    
    # Save plot
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plot_file = figures_dir / f"coordination_analysis_{timestamp}.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"📊 Coordination analysis plot saved: {plot_file}")
    return plot_file

if 'df' in locals() and not df.empty:
    plot_coordination_analysis(df)
else:
    print("❌ No data available for plotting")


In [ ]:
# %% Create Summary Statistics Plot
def plot_summary_statistics(df):
    """Create summary statistics and distributions plot"""
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: TMT distribution before vs after
    ax1.hist(df['tmt_before'], bins=20, alpha=0.7, label='TMT Antes', 
             color='#e74c3c', density=True, edgecolor='black')
    ax1.hist(df['tmt_after'], bins=20, alpha=0.7, label='TMT Después', 
             color='#3498db', density=True, edgecolor='black')
    ax1.set_xlabel('TMT (segundos)')
    ax1.set_ylabel('Densidad')
    ax1.set_title('Distribución TMT: Antes vs Después', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: TMT improvement distribution
    ax2.hist(df['tmt_improvement'], bins=20, alpha=0.7, color='#2ecc71', 
             density=True, edgecolor='black')
    ax2.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Sin Cambio')
    ax2.set_xlabel('Mejora TMT (segundos)')
    ax2.set_ylabel('Densidad')
    ax2.set_title('Distribución de Mejora TMT', fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Coordination percentage distribution
    ax3.hist(df['coordination_pct_before'], bins=20, alpha=0.7, 
             label='Coord % Antes', color='#e67e22', density=True, edgecolor='black')
    ax3.hist(df['coordination_pct_after'], bins=20, alpha=0.7, 
             label='Coord % Después', color='#9b59b6', density=True, edgecolor='black')
    ax3.set_xlabel('Porcentaje de Coordinación (%)')
    ax3.set_ylabel('Densidad')
    ax3.set_title('Distribución de Coordinación', fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Improvement percentage distribution
    ax4.hist(df['improvement_percentage'], bins=20, alpha=0.7, color='#f39c12', 
             density=True, edgecolor='black')
    ax4.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Sin Cambio')
    ax4.set_xlabel('Porcentaje de Mejora (%)')
    ax4.set_ylabel('Densidad')
    ax4.set_title('Distribución de Porcentaje de Mejora', fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save plot
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plot_file = figures_dir / f"summary_statistics_{timestamp}.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"📊 Summary statistics plot saved: {plot_file}")
    return plot_file

if 'df' in locals() and not df.empty:
    plot_summary_statistics(df)
else:
    print("❌ No data available for plotting")


In [ ]:
# %% Create Detailed TMT Table with Values
def create_detailed_tmt_table(df):
    """Create detailed table showing TMT values after optimization"""
    
    # Create enhanced table with all important columns
    enhanced_df = df.copy()
    
    # Add status column
    enhanced_df['status'] = enhanced_df['tmt_improvement'].apply(
        lambda x: '✅ MEJORÓ' if x > 0 else '❌ EMPEORÓ' if x < 0 else '➖ SIN CAMBIO'
    )
    
    # Add category column
    def categorize_performance(improvement):
        if improvement > 1:
            return 'Excelente'
        elif improvement > 0:
            return 'Bueno'
        elif improvement > -1:
            return 'Neutral'
        elif improvement > -3:
            return 'Malo'
        else:
            return 'Muy Malo'
    
    enhanced_df['categoria'] = enhanced_df['tmt_improvement'].apply(categorize_performance)
    
    # Select and reorder columns for better display
    display_columns = [
        'scenario_id', 'status', 'categoria',
        'tmt_before', 'tmt_after', 'tmt_improvement', 'improvement_percentage',
        'coordination_pct_before', 'coordination_pct_after',
        'total_pairs', 'coordinated_before', 'coordinated_after'
    ]
    
    display_df = enhanced_df[display_columns].copy()
    
    # Rename columns for better display
    display_df.columns = [
        'Escenario', 'Estado', 'Categoría',
        'TMT Antes', 'TMT Después', 'Mejora TMT', '% Mejora',
        'Coord % Antes', 'Coord % Después',
        'Total Pares', 'Coord Antes', 'Coord Después'
    ]
    
    print("📊 TABLA DETALLADA DE TMT DESPUÉS DE OPTIMIZACIÓN")
    print("="*120)
    print("🎯 INTERPRETACIÓN:")
    print("   • TMT = Total Miscoordination Time (tiempo total de descoordinación)")
    print("   • TMT Después = Valor de TMT después de la optimización GA")
    print("   • Mejora TMT = TMT_antes - TMT_después (positivo = mejora)")
    print("   • Coord % = Porcentaje de pares coordinados")
    print("="*120)
    
    # Display top 20 results
    print("\n🏆 TOP 20 RESULTADOS:")
    display(display_df.head(20))
    
    # Display summary statistics
    print(f"\n📈 ESTADÍSTICAS RESUMEN:")
    print(f"   • Total escenarios: {len(display_df)}")
    print(f"   • Escenarios que mejoraron: {len(display_df[display_df['Mejora TMT'] > 0])}")
    print(f"   • Escenarios que empeoraron: {len(display_df[display_df['Mejora TMT'] < 0])}")
    print(f"   • TMT total antes: {display_df['TMT Antes'].sum():.3f} segundos")
    print(f"   • TMT total después: {display_df['TMT Después'].sum():.3f} segundos")
    print(f"   • Cambio total TMT: {display_df['Mejora TMT'].sum():+.3f} segundos")
    
    # Show best and worst performers
    print(f"\n🎯 MEJORES RESULTADOS (MAYOR REDUCCIÓN DE TMT):")
    best_5 = display_df.head(5)
    for i, (_, row) in enumerate(best_5.iterrows(), 1):
        print(f"   {i}. {row['Escenario']}: {row['TMT Antes']:.3f}s → {row['TMT Después']:.3f}s "
              f"(Mejora: {row['Mejora TMT']:+.3f}s, {row['% Mejora']:+.1f}%)")
    
    print(f"\n📉 PEORES RESULTADOS (MAYOR AUMENTO DE TMT):")
    worst_5 = display_df.tail(5)
    for i, (_, row) in enumerate(worst_5.iterrows(), 1):
        print(f"   {i}. {row['Escenario']}: {row['TMT Antes']:.3f}s → {row['TMT Después']:.3f}s "
              f"(Cambio: {row['Mejora TMT']:+.3f}s, {row['% Mejora']:+.1f}%)")
    
    return display_df

if 'df' in locals() and not df.empty:
    detailed_table = create_detailed_tmt_table(df)
else:
    print("❌ No data available for table creation")


In [ ]:
# %% Save Results and Export Data
def save_results_and_export(df):
    """Save all results and export data"""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. Save enhanced DataFrame as CSV
    csv_file = processed_dir / f"tmt_analysis_with_values_{timestamp}.csv"
    df.to_csv(csv_file, index=False, encoding='utf-8')
    print(f"📊 Enhanced TMT data saved: {csv_file}")
    
    # 2. Create summary report
    report_file = processed_dir / f"tmt_summary_report_{timestamp}.txt"
    
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write("REPORTE RESUMEN - ANÁLISIS TMT DESPUÉS DE OPTIMIZACIÓN GA\n")
        f.write("="*80 + "\n")
        f.write(f"Generado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Total Escenarios: {len(df)}\n\n")
        
        f.write("ESTADÍSTICAS GENERALES:\n")
        f.write("-" * 40 + "\n")
        f.write(f"Escenarios que mejoraron: {len(df[df['tmt_improvement'] > 0])}\n")
        f.write(f"Escenarios que empeoraron: {len(df[df['tmt_improvement'] < 0])}\n")
        f.write(f"TMT total antes: {df['tmt_before'].sum():.3f} segundos\n")
        f.write(f"TMT total después: {df['tmt_after'].sum():.3f} segundos\n")
        f.write(f"Cambio total TMT: {df['tmt_improvement'].sum():+.3f} segundos\n\n")
        
        f.write("TOP 10 MEJORES RESULTADOS:\n")
        f.write("-" * 40 + "\n")
        for i, (_, row) in enumerate(df.head(10).iterrows(), 1):
            f.write(f"{i:2d}. {row['scenario_id']}: {row['tmt_before']:.3f}s → {row['tmt_after']:.3f}s "
                   f"(Mejora: {row['tmt_improvement']:+.3f}s, {row['improvement_percentage']:+.1f}%)\n")
        
        f.write("\nTOP 10 PEORES RESULTADOS:\n")
        f.write("-" * 40 + "\n")
        for i, (_, row) in enumerate(df.tail(10).iterrows(), 1):
            f.write(f"{i:2d}. {row['scenario_id']}: {row['tmt_before']:.3f}s → {row['tmt_after']:.3f}s "
                   f"(Cambio: {row['tmt_improvement']:+.3f}s, {row['improvement_percentage']:+.1f}%)\n")
    
    print(f"📝 Summary report saved: {report_file}")
    
    # 3. Display file locations
    print(f"\n📁 ARCHIVOS GENERADOS:")
    print("="*60)
    print(f"1. 📊 Datos CSV: {csv_file.name}")
    print(f"2. 📝 Reporte: {report_file.name}")
    print(f"3. 📈 Gráficas: {figures_dir}")
    
    return csv_file, report_file

if 'df' in locals() and not df.empty:
    csv_file, report_file = save_results_and_export(df)
    print("\n✅ Análisis TMT con gráficas completado exitosamente!")
else:
    print("❌ No data available for saving")
